# Phylodiversity data collection

We are going to use three different data sources to get data: BOLD, GenBank (Using GeneDumper) and ButterflyNet. Some sequences may be novelly sequenced. 

## ButterflyNet Data

Per Caroline there are four plates that have been sequenced and cleaned on the cluster - 

FMN_456206_Plates 7-14 13 loci
/orange/kawahara/RAPiD/FMN_456206/IBA/SPLIT

FMN_456204_p5p7 450 loc
/orange/kawahara/RAPiD/FMN_456204/FMN_456204_p5p7/IBA/SPLIT

FMN_456204_P2_P4 450 loci
/ufrc/kawahara/share/BUTTERFLYNET_DATA/FMN_456204_P2_P4/IBA/SPLIT

/ufrc/kawahara/share/BUTTERFLYNET_DATA/FMN_456206/ALL_Plates/IBA/SPLIT

These were downloaded and are named: 

* sl_FULLASS_CLEAN_13loci1.fa
* sl_FULLASS_CLEAN_13loci2.fa
* sl_FULLASS_CLEAN_450loci1.fa
* sl_FULLASS_CLEAN_450loci2.fa

The four files were concatenated together and loci split out into individual files using split.py

In [9]:
cat *.fa > all_loci.fa

./split.py all_loci.fa _

cat: all_loci.fa: input file is output file


Since we only want the first 13 loci, we delete the others.

In [15]:
rm L{14..423}.fa

rm: cannot remove ‘L392.fa’: No such file or directory
rm: cannot remove ‘L393.fa’: No such file or directory
rm: cannot remove ‘L394.fa’: No such file or directory
rm: cannot remove ‘L395.fa’: No such file or directory
rm: cannot remove ‘L396.fa’: No such file or directory
rm: cannot remove ‘L397.fa’: No such file or directory
rm: cannot remove ‘L398.fa’: No such file or directory
rm: cannot remove ‘L399.fa’: No such file or directory
rm: cannot remove ‘L400.fa’: No such file or directory
rm: cannot remove ‘L401.fa’: No such file or directory
rm: cannot remove ‘L402.fa’: No such file or directory
rm: cannot remove ‘L403.fa’: No such file or directory
rm: cannot remove ‘L404.fa’: No such file or directory
rm: cannot remove ‘L406.fa’: No such file or directory
rm: cannot remove ‘L407.fa’: No such file or directory
rm: cannot remove ‘L408.fa’: No such file or directory
rm: cannot remove ‘L409.fa’: No such file or directory
rm: cannot remove ‘L410.fa’: No such file or directory
rm: cannot

: 1

In [23]:
ls

all_loci.fa  L1.fa  L6.fa                        sl_FULLASS_CLEAN_13loci2.fa
L10.fa       L2.fa  L7.fa                        sl_FULLASS_CLEAN_450loci1.fa
L11.fa       L3.fa  L8.fa                        sl_FULLASS_CLEAN_450loci2.fa
L12.fa       L4.fa  L9.fa                        split.py
L13.fa       L5.fa  sl_FULLASS_CLEAN_13loci1.fa


Next we have to rename the sequences with a custom python script - this will rename them to genus_species

In [6]:
for i in L*.fa; do python rename_seqs.py -i $i -t BN; done

In [9]:
ls -t

L9_renamed.fas  L13_renamed.fas  L9.fa  all_loci.fa
L8_renamed.fas  L12_renamed.fas  L8.fa  sl_FULLASS_CLEAN_450loci1.fa
L7_renamed.fas  L11_renamed.fas  L1.fa  sl_FULLASS_CLEAN_13loci1.fa
L6_renamed.fas  L10_renamed.fas  L3.fa  sl_FULLASS_CLEAN_13loci2.fa
L5_renamed.fas  rename_seqs.py   L2.fa  sl_FULLASS_CLEAN_450loci2.fa
L4_renamed.fas  L13.fa           L5.fa  split.py
L3_renamed.fas  L12.fa           L4.fa
L2_renamed.fas  L11.fa           L7.fa
L1_renamed.fas  L10.fa           L6.fa


Next we need to remove duplicates (and in doing so turn it into single lines)

In [15]:
for i in L*renamed.fas; do awk '{if(NR==1) {print $0} else {if($0 ~ /^>/) {print "\n"$0} else {printf $0}}}' $i > $i\_sl.fa; done

In [20]:
for i in L*_sl.fa; do awk 'BEGIN{RS=">"}NR>1{sub("\n","\t"); gsub("\n",""); print RS$0}' $i | awk '!seen[$1]++' | awk -v OFS="\n" '{print $1,$2}' > $i\_nodups.fa; done

Code below not for this analysis but removes duplicates and keeps none of them. Above keeps the first sequence it finds.

In [ ]:
#for i in AA_L*.fas; do awk 'BEGIN{RS=">"}NR>1{sub("\n","\t"); gsub("\n",""); print RS$0}' $i | sort -k1 | awk '{c[$1]++; t[$1]=$0} END {for(k in c) {if (c[k]==1) print t[k]}}' | awk -v OFS="\n" '{print $1,$2}' > $(basename "$i" .fas)\_nodups.fas; done

We also need to pull only the North American species using a novel script

In [22]:
for i in L*_nodups.fa; do python pullseqs.py $i BNet_species.csv; done

In [23]:
mkdir NA_sequences
mv *_NA.fa NA_sequences

In [24]:
cd NA_sequences
ls

L10_renamed.fas_sl.fa_nodups.fa_NA.fa  L4_renamed.fas_sl.fa_nodups.fa_NA.fa
L11_renamed.fas_sl.fa_nodups.fa_NA.fa  L5_renamed.fas_sl.fa_nodups.fa_NA.fa
L12_renamed.fas_sl.fa_nodups.fa_NA.fa  L6_renamed.fas_sl.fa_nodups.fa_NA.fa
L13_renamed.fas_sl.fa_nodups.fa_NA.fa  L7_renamed.fas_sl.fa_nodups.fa_NA.fa
L1_renamed.fas_sl.fa_nodups.fa_NA.fa   L8_renamed.fas_sl.fa_nodups.fa_NA.fa
L2_renamed.fas_sl.fa_nodups.fa_NA.fa   L9_renamed.fas_sl.fa_nodups.fa_NA.fa
L3_renamed.fas_sl.fa_nodups.fa_NA.fa


In [25]:
mv L1_renamed.fas_sl.fa_nodups.fa_NA.fa COI_BN.fa
mv L2_renamed.fas_sl.fa_nodups.fa_NA.fa CoA_BN.fa
mv L3_renamed.fas_sl.fa_nodups.fa_NA.fa CAD_BN.fa
mv L4_renamed.fas_sl.fa_nodups.fa_NA.fa CAT_BN.fa
mv L5_renamed.fas_sl.fa_nodups.fa_NA.fa DDC_BN.fa
mv L6_renamed.fas_sl.fa_nodups.fa_NA.fa Ef-1a_BN.fa
mv L7_renamed.fas_sl.fa_nodups.fa_NA.fa GAPDH_BN.fa
mv L8_renamed.fas_sl.fa_nodups.fa_NA.fa HCL_BN.fa
mv L9_renamed.fas_sl.fa_nodups.fa_NA.fa IDH_BN.fa
mv L10_renamed.fas_sl.fa_nodups.fa_NA.fa MDH_BN.fa
mv L11_renamed.fas_sl.fa_nodups.fa_NA.fa RpS2_BN.fa
mv L12_renamed.fas_sl.fa_nodups.fa_NA.fa RpS5_BN.fa
mv L13_renamed.fas_sl.fa_nodups.fa_NA.fa WGL_BN.fa

For some reason one certain Species had duplicate sequences, so I went through and deleted each one.

Now we need to align them using a script. This script splits the sequences up by length, aligns the longer ones and then adds the shorter ones using --addfrags in mafft

In [7]:
for i in *.fa; do python LONGREF_ALIGN.py $i 2> /dev/null; done

CAD_BN_FULL.fas:215 seqs
CAD_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4
CAT_BN_FULL.fas:1 seqs
CAT_BN_FRAGS.fas: 200 seqs

RUNNING 1 FULL, LOTS FRAGS
CoA_BN_FULL.fas:44 seqs
CoA_BN_FRAGS.fas: 150 seqs

RUNNING FEW FULL, LOTS FRAGS
COI_BN_FULL.fas:1 seqs
COI_BN_FRAGS.fas: 209 seqs

RUNNING 1 FULL, LOTS FRAGS
DDC_BN_FULL.fas:86 seqs
DDC_BN_FRAGS.fas: 122 seqs

RUNNING LOTS FULL, LOTS FRAGS
0
1
0
1
2
Ef-1a_BN_FULL.fas:220 seqs
Ef-1a_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4
GAPDH_BN_FULL.fas:209 seqs
GAPDH_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4
HCL_BN_FULL.fas:208 seqs
HCL_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4
IDH_BN_FULL.fas:22 seqs
IDH_BN_FRAGS.fas: 187 seqs

RUNNING FEW FULL, LOTS FRAGS
MDH_BN_FULL.fas:211 seqs
MDH_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4
RpS2_BN_FULL.fas:202 seqs
RpS2_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4
RpS5_BN_FULL.fas:205 seqs
RpS5_BN_FRAGS.fas: 0 seqs


Now we're going to move each final alignment to a new file

In [8]:
mkdir aligned_files
mv WGL_BN_FINALALIGN.fa aligned_files
mv RpS5_BN_aligned5.fas aligned_files
mv RpS2_BN_aligned5.fas aligned_files
mv MDH_BN_aligned5.fas aligned_files
mv IDH_BN_aligned4.fas aligned_files
mv HCL_BN_aligned5.fas aligned_files
mv GAPDH_BN_aligned5.fas aligned_files
mv Ef-1a_BN_aligned5.fas aligned_files
mv DDC_BN_FINALALIGN.fa aligned_files
mv COI_BN_aligned5.fas aligned_files
mv CoA_BN_aligned3.fas aligned_files
mv CAT_BN_aligned4.fas aligned_files
mv CAD_BN_aligned5.fas aligned_files

CAT doesn't look like it aligned correctly, so I'm going to go back to the original sequences, shuffle them in aliview and realign.

In [9]:
rm CAT_BN_*

I used Aliview to sort by sequence name and then resaved it

In [10]:
python LONGREF_ALIGN.py CAT_BN.fa 2> /dev/null

CAT_BN_FULL.fas:201 seqs
CAT_BN_FRAGS.fas: 0 seqs

RUNNING LOTS FULL, 0 FRAGS
0
1
2
3
4


In [11]:
mv CAT_BN_aligned5.fas aligned_files

That worked, it looks better. Now since there are flanking regions, I'm going to use a threshold to remove sites that are 10% or more gaps and then realign using mafft.

In [12]:
cd aligned_files

In [14]:
for i in *.fas; do python remove_sites_threshold.py -i $i -t .1; done

In [5]:
for i in *10.fas; do mafft $i > ${i/%.fas/_aligned.fas} 2> /dev/null; done

In [6]:
mkdir aligned_and_threshold
mv *_aligned.fas aligned_and_threshold

Next I'm going to use Aliview to translate to amino acids, find the reading frame, and chop off the extra flanks where the 'messiness' starts. I'm being conservative about chopping, since when these get aligned to longer sequences, we can clean these up more.

The last step is to rename them again since mafft adds prefixes to sequences that have been reversed.

In [2]:
for i in *.fas; do sed -i 's/_R_//g' $i; done

In [52]:
mv CAD_BN_aligned5_threshold10_aligned.fas CAD_BN.fa
mv CAT_BN_aligned5_threshold10_aligned.fas CAT_BN.fa
mv CoA_BN_aligned3_threshold10_aligned.fas CoA_BN.fa
mv COI_BN_aligned5_threshold10_aligned.fas COI_trnL_COII_BN.fa
mv DDC_BN_FINALALIGN_threshold10_aligned.fas DDC_BN.fa
mv Ef-1a_BN_aligned5_threshold10_aligned.fas Ef-1a_BN.fa
mv GAPDH_BN_aligned5_threshold10_aligned.fas GAPDH_BN.fa
mv HCL_BN_aligned5_threshold10_aligned.fas HCL_BN.fa
mv IDH_BN_aligned4_threshold10_aligned.fas IDH_BN.fa
mv MDH_BN_aligned5_threshold10_aligned.fas MDH_BN.fa
mv RpS2_BN_aligned5_threshold10_aligned.fas RpS2_BN.fa
mv RpS5_BN_aligned5_threshold10_aligned.fas RpS5_BN.fa
mv WGL_BN_FINALALIGN_threshold10_aligned.fas WGL_BN.fa

And we're done!!

## BOLD Data

In [7]:
mkdir BOLD
cd BOLD

We use a novel script to hit the BOLD API for each name in our North American list. This downloads the sequences for all the potential loci for us into individual fasta files based on how they are named.

In [4]:
python BOLD_query.py

In [5]:
ls

16S.fa         CAD.fa     EF1.fa    ITS.fa       ND3.fa   rename_seqs.py
18S.fa         COI.fa     fbpA.fa   MDH.fa       ND4.fa   RpS2.fa
ArgKin.fa      COII.fa    GAPDH.fa  NAnames.csv  ND4L.fa  RpS5.fa
atp6.fa        COXIII.fa  IDH.fa    ND1.fa       ND5.fa   Wnt1.fa
BOLD_query.py  CYTB.fa    ITS2.fa   ND2.fa       ND6.fa


We make a new directory, move only the ones we are interested in and remove the others

In [6]:
mkdir BOLD_seqs
mv -t BOLD_seqs ArgKin.fa CAD.fa COI.fa COII.fa EF1.fa GAPDH.fa IDH.fa MDH.fa RpS2.fa RpS5.fa Wnt1.fa

In [7]:
cd BOLD_seqs

I'm going to kep COI and COII apart for now (they will not align correctly because there is no full length sequence currently), rename Wnt1 to Wng to keep names the same throughout, and add a BOLD suffix to the end of each file

In [8]:
mv COI.fa COI_BOLD.fa
mv COII.fa COII_BOLD.fa
mv Wnt1.fa Wng_BOLD.fa
mv ArgKin.fa ArgKin_BOLD.fa
mv CAD.fa CAD_BOLD.fa
mv EF1.fa Ef-1a_BOLD.fa
mv GAPDH.fa GAPDH_BOLD.fa
mv IDH.fa IDH_BOLD.fa
mv MDH.fa MDH_BOLD.fa
mv RpS2.fa RpS2_BOLD.fa
mv RpS5.fa RpS5_BOLD.fa

Now, same as before, I'm going to rename the sequences using a custom python script. There are no duplicates to remove.

In [10]:
for i in *.fa; do python rename_seqs.py -i $i -t BOLD; done

Now to align them using the same script as above

In [12]:
mkdir align_files
mv *_renamed.fas align_files

In [14]:
cd align_files
for i in *.fas; do python LONGREF_ALIGN.py $i 2> /dev/null; done

ArgKin_BOLD_renamed_FULL.fas:23 seqs
ArgKin_BOLD_renamed_FRAGS.fas: 4 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT.....
RUNNING MAFFT ADDING FRAGS.....
CAD_BOLD_renamed_FULL.fas:36 seqs
CAD_BOLD_renamed_FRAGS.fas: 6 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT.....
RUNNING MAFFT ADDING FRAGS.....
COI_BOLD_renamed_FULL.fas:4 seqs
COI_BOLD_renamed_FRAGS.fas: 1185 seqs

RUNNING FEW FULL, LOTS FRAGS
COII_BOLD_renamed_FULL.fas:9 seqs
COII_BOLD_renamed_FRAGS.fas: 2 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT.....
RUNNING MAFFT ADDING FRAGS.....
Ef-1a_BOLD_renamed_FULL.fas:64 seqs
Ef-1a_BOLD_renamed_FRAGS.fas: 10 seqs

RUNNING LOTS FULL, FEW FRAGS
GAPDH_BOLD_renamed_FULL.fas:35 seqs
GAPDH_BOLD_renamed_FRAGS.fas: 5 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT.....
RUNNING MAFFT ADDING FRAGS.....
IDH_BOLD_renamed_FULL.fas:30 seqs
IDH_BOLD_renamed_FRAGS.fas: 3 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT..

: 1

Now we're going to move each final alignment to a new file and check the alignments

In [16]:
mkdir aligned_files
mv Wng_BOLD_renamed_aligned2.fas aligned_files
mv RpS5_BOLD_renamed_FINALALIGN.fa aligned_files
mv RpS2_BOLD_renamed_FINALALIGN.fa aligned_files
mv MDH_BOLD_renamed_FINALALIGN.fa aligned_files
mv IDH_BOLD_renamed_FINALALIGN.fa aligned_files
mv GAPDH_BOLD_renamed_FINALALIGN.fa aligned_files
mv Ef-1a_BOLD_renamed_aligned2.fas aligned_files
mv COII_BOLD_renamed_FINALALIGN.fa aligned_files
mv COI_BOLD_renamed_aligned24.fas aligned_files
mv CAD_BOLD_renamed_FINALALIGN.fa aligned_files
mv ArgKin_BOLD_renamed_FINALALIGN.fa aligned_files

COI aligned incorrectly. I'm going to take the sequences that didn't align(need a program to do this), align each group separately and removed inserts, then try and align them together.

program? - https://www.biostars.org/p/4855/

In [6]:
mkdir COI_align
mv COI_BOLD_renamed_aligned24.fas COI_align

Moved COI back to main alignment folder

In [8]:
ls

ArgKin_BOLD_renamed_FINALALIGN.fa
CAD_BOLD_renamed_FINALALIGN.fa
COI_align
COI_BOLD_renamed_aligned24_aligned_threshold90.fas
COII_BOLD_renamed_FINALALIGN.fa
Ef-1a_BOLD_renamed_aligned2.fas
GAPDH_BOLD_renamed_FINALALIGN.fa
IDH_BOLD_renamed_FINALALIGN.fa
MDH_BOLD_renamed_FINALALIGN.fa
RpS2_BOLD_renamed_FINALALIGN.fa
RpS5_BOLD_renamed_FINALALIGN.fa
Wng_BOLD_renamed_aligned2.fas


That worked, it looks better. Now since there are inserts, I'm going to use a threshold to remove sites that are 90% or more gaps (because not all sequences are full length) and then realign using mafft.

In [13]:
for i in *.fa*; do python remove_sites_threshold.py -i $i -t .9; done

In [22]:
for i in *90.fas; do mafft $i > ${i/%.fas/_aligned.fas} 2> /dev/null; done

Next I'm going to use Aliview to translate to amino acids, find the reading frame, and chop off the extra flanks where the 'messiness' starts. I'm being conservative about chopping, since when these get aligned to longer sequences, we can clean these up more.

The last step is to rename them again since mafft adds prefixes to sequences that have been reversed. - There are none :)

There's a few COI sequences that are very uninformative and contain a lot of Ns - this is messing up the alignement and will eventually mess up any analyses we do. So I will remove them.

Chlosyne_fulvia
Chlosyne_californica
Xeniades_orchamus
Erebia_dabanensis

Now to move them to their own folder and rename them!

In [10]:
mkdir final_align

In [11]:
mv *_edit.fas final_align

In [12]:
cd final_align

In [13]:
mv ArgKin_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas ArgKin_BOLD.fas
mv CAD_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas CAD_BOLD.fas
mv COI_BOLD_renamed_aligned24_aligned_threshold90_aligned_edit.fas COI_BOLD.fas
mv COII_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas COII_BOLD.fas
mv Ef-1a_BOLD_renamed_aligned2_threshold90_aligned_edit.fas Ef-1a_BOLD.fas
mv GAPDH_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas GAPDH_BOLD.fas
mv IDH_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas IDH_BOLD.fas
mv MDH_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas MDH_BOLD.fas
mv RpS2_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas RpS2_BOLD.fas
mv RpS5_BOLD_renamed_FINALALIGN_threshold90_aligned_edit.fas RpS5_BOLD.fas
mv Wng_BOLD_renamed_aligned2_threshold90_aligned_edit.fas Wgl_BOLD.fas

Done!!

## GenBank Data

I'm going to use GeneDumper to pull down and clean sequences. GeneDumper was pulled down from https://github.com/sunray1/GeneDumper on 7/8/19. I know there are issues with the ncbi_check part. (7/9 - this has been fixed)

In [19]:
mkdir GenBank
cd GenBank

In [20]:
git clone https://github.com/sunray1/GeneDumper

Cloning into 'GeneDumper'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 231 (delta 5), reused 0 (delta 0), pack-reused 217
Receiving objects: 100% (231/231), 10.86 MiB | 193.00 KiB/s, done.
Resolving deltas: 100% (97/97), done.
Checking connectivity... done.


In [21]:
cd GeneDumper

I've already made the taxonomy into a SQL database and I have the fasta file with 14 genes of interest. This has been copied over.

In [23]:
ls

alignlib  D_plexippus_probes.fas  GeneDump.py               README.md
blastlib  example_files           __init__.py               taxonomy_files
cleanlib  GeneClean.py            NAButterflies_wsyns_2.db


In [25]:
./GeneDump.py -s 0123 -t NAButterflies_wsyns_2.db -f D_plexippus_probes.fas

Running step 0: run blast

Blasting ArgKin
Blasting RpS2
Blasting IDH
Blasting Wgl
Blasting RpS5
Blasting MDH
Blasting HCL
Blasting GAPDH
Blasting EF-1a
Blasting DDC
Blasting CoA
Blasting CAT
Blasting CAD
Blasting COI_trnL_COII
Step 0 completed
--------------------
Running step 1: converting .xml to .csv.....

Converting ArgKin.xml...
ArgKin.xml converted

Converting CAD.xml...
CAD.xml converted

Converting CAT.xml...
CAT.xml converted

Converting CoA.xml...
CoA.xml converted

Converting COI_trnL_COII.xml...
COI_trnL_COII.xml converted

Converting DDC.xml...
DDC.xml converted

Converting EF-1a.xml...
EF-1a.xml converted

Converting GAPDH.xml...
GAPDH.xml converted

Converting HCL.xml...
HCL.xml converted

Converting IDH.xml...
IDH.xml converted

Converting MDH.xml...
MDH.xml converted

Converting RpS2.xml...
RpS2.xml converted

Converting RpS5.xml...
RpS5.xml converted

Converting Wgl.xml...
Wgl.xml converted

Step 1 completed
----------------------
Running step 2: converting .csv to .

: 1

I redid RsS5 and MDH because it seemed like they didn't blast correctly.

Now to continue to try and resolve names

In [29]:
./GeneDump.py -s 456 -t NAButterflies_wsyns_2.db

Running step 4: checking ncbi for names.....

12
1
parsing records
finished parsing
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
2
parsing records
finished parsing
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
3
parsing records
finished parsing
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating records
updating reco

I removed the 'Predicted' sequences from the sql database by hand and now I'm going to go through the spelling output to make sure that all spellings changed are correct and change them back if they are not.

Done! Now I'm going to get the presence/absense matrix for a quick glance at the data.

In [5]:
./GeneDump.py -c Species -t NAButterflies_wsyns_2.db

Calculating statistics for Species level.....

Calculating statistics completed
--------------------
Done


Now to start cleaning! Using GeneClean.py, I'm going to try and run the whole thing through. I may have to split this up depending on how long this takes.

In [6]:
mkdir GeneDump_outputs
mv *.txt GeneDump_outputs
mv *.csv GeneDump_outputs
mv *.xml GeneDump_outputs

In [7]:
./GeneClean.py -t NAButterflies_wsyns_2.db

Running initial resolver
Trying to resolve COI/COII sequences
1057_COI_trnL_COII
1
0.11%
1712_COI_trnL_COII
2
0.22%
1562_COI_trnL_COII
3
0.33%
45_COI_trnL_COII
4
0.44%
136_COI_trnL_COII
5
0.55%
1602_COI_trnL_COII
6
0.65%
1394_COI_trnL_COII
7
0.76%
1917_COI_trnL_COII
8
0.87%
596_COI_trnL_COII
9
0.98%
2036_COI_trnL_COII
10
1.09%
570_COI_trnL_COII
11
1.2%
2425_COI_trnL_COII
12
1.31%
1295_COI_trnL_COII
13
1.42%
1731_COI_trnL_COII
14
1.53%
62_COI_trnL_COII
15
1.64%
259_COI_trnL_COII
16
1.74%
2194_COI_trnL_COII
17
1.85%
1303_COI_trnL_COII
18
1.96%
200_COI_trnL_COII
19
2.07%
339_COI_trnL_COII
20
2.18%
2466_COI_trnL_COII
21
2.29%
1069_COI_trnL_COII
22
2.4%
1836_COI_trnL_COII
23
2.51%
46_COI_trnL_COII
24
2.62%
1751_COI_trnL_COII
25
2.73%
1522_COI_trnL_COII
26
2.84%
1285_COI_trnL_COII
27
2.94%
1482_COI_trnL_COII
28
3.05%
92_COI_trnL_COII
29
3.16%
943_COI_trnL_COII
30
3.27%
2293_COI_trnL_COII
31
3.38%
69_COI_trnL_COII
32
3.49%
125_COI_trnL_COII
33
3.6%
755_COI_trnL_COII
34
3.71%
1478_COI_trnL_COI

283
30.86%
333_COI_trnL_COII
284
30.97%
951_COI_trnL_COII
285
31.08%
2095_COI_trnL_COII
286
31.19%
1668_COI_trnL_COII
287
31.3%
1367_COI_trnL_COII
288
31.41%
1372_COI_trnL_COII
289
31.52%
403_COI_trnL_COII
290
31.62%
511_COI_trnL_COII
291
31.73%
2166_COI_trnL_COII
292
31.84%
432_COI_trnL_COII
293
31.95%
341_COI_trnL_COII
294
32.06%
816_COI_trnL_COII
295
32.17%
1358_COI_trnL_COII
296
32.28%
469_COI_trnL_COII
297
32.39%
2452_COI_trnL_COII
298
32.5%
1249_COI_trnL_COII
299
32.61%
272_COI_trnL_COII
300
32.72%
986_COI_trnL_COII
301
32.82%
1368_COI_trnL_COII
302
32.93%
1362_COI_trnL_COII
303
33.04%
52_COI_trnL_COII
304
33.15%
1618_COI_trnL_COII
305
33.26%
1676_COI_trnL_COII
306
33.37%
372_COI_trnL_COII
307
33.48%
1606_COI_trnL_COII
308
33.59%
1434_COI_trnL_COII
309
33.7%
2446_COI_trnL_COII
310
33.81%
544_COI_trnL_COII
311
33.91%
524_COI_trnL_COII
312
34.02%
554_COI_trnL_COII
313
34.13%
888_COI_trnL_COII
314
34.24%
662_COI_trnL_COII
315
34.35%
1673_COI_trnL_COII
316
34.46%
594_COI_trnL_COII
31

61.18%
2378_COI_trnL_COII
562
61.29%
385_COI_trnL_COII
563
61.4%
1556_COI_trnL_COII
564
61.5%
598_COI_trnL_COII
565
61.61%
56_COI_trnL_COII
566
61.72%
820_COI_trnL_COII
567
61.83%
1071_COI_trnL_COII
568
61.94%
1204_COI_trnL_COII
569
62.05%
1632_COI_trnL_COII
570
62.16%
872_COI_trnL_COII
571
62.27%
1419_COI_trnL_COII
572
62.38%
1157_COI_trnL_COII
573
62.49%
1408_COI_trnL_COII
574
62.6%
520_COI_trnL_COII
575
62.7%
671_COI_trnL_COII
576
62.81%
1601_COI_trnL_COII
577
62.92%
1254_COI_trnL_COII
578
63.03%
109_COI_trnL_COII
579
63.14%
918_COI_trnL_COII
580
63.25%
2237_COI_trnL_COII
581
63.36%
1920_COI_trnL_COII
582
63.47%
729_COI_trnL_COII
583
63.58%
2352_COI_trnL_COII
584
63.69%
844_COI_trnL_COII
585
63.79%
1497_COI_trnL_COII
586
63.9%
1579_COI_trnL_COII
587
64.01%
1875_COI_trnL_COII
588
64.12%
1960_COI_trnL_COII
589
64.23%
24_COI_trnL_COII
590
64.34%
2326_COI_trnL_COII
591
64.45%
1874_COI_trnL_COII
592
64.56%
541_COI_trnL_COII
593
64.67%
160_COI_trnL_COII
594
64.78%
49_COI_trnL_COII
595
64.

1648_COI_trnL_COII
840
91.6%
1247_COI_trnL_COII
841
91.71%
1326_COI_trnL_COII
842
91.82%
1638_COI_trnL_COII
843
91.93%
1279_COI_trnL_COII
844
92.04%
1230_COI_trnL_COII
845
92.15%
2468_COI_trnL_COII
846
92.26%
1171_COI_trnL_COII
847
92.37%
1038_COI_trnL_COII
848
92.48%
807_COI_trnL_COII
849
92.58%
734_COI_trnL_COII
850
92.69%
2207_COI_trnL_COII
851
92.8%
2429_COI_trnL_COII
852
92.91%
1239_COI_trnL_COII
853
93.02%
1998_COI_trnL_COII
854
93.13%
967_COI_trnL_COII
855
93.24%
886_COI_trnL_COII
856
93.35%
1935_COI_trnL_COII
857
93.46%
1421_COI_trnL_COII
858
93.57%
1932_COI_trnL_COII
859
93.68%
1940_COI_trnL_COII
860
93.78%
673_COI_trnL_COII
861
93.89%
1319_COI_trnL_COII
862
94.0%
721_COI_trnL_COII
863
94.11%
1774_COI_trnL_COII
864
94.22%
1508_COI_trnL_COII
865
94.33%
2260_COI_trnL_COII
866
94.44%
1439_COI_trnL_COII
867
94.55%
288_COI_trnL_COII
868
94.66%
1805_COI_trnL_COII
869
94.77%
1260_COI_trnL_COII
870
94.87%
687_COI_trnL_COII
871
94.98%
2027_COI_trnL_COII
872
95.09%
1501_COI_trnL_COII
87

71.87%
71.95%
72.03%
72.12%
72.2%
72.28%
72.37%
72.45%
72.53%
72.61%
72.7%
72.78%
72.86%
72.95%
73.03%
73.11%
73.2%
73.28%
73.36%
73.44%
73.53%
73.61%
73.69%
73.78%
73.86%
73.94%
74.02%
74.11%
74.19%
74.27%
74.36%
74.44%
74.52%
74.61%
74.69%
74.77%
74.85%
74.94%
75.02%
75.1%
75.19%
75.27%
75.35%
75.44%
75.52%
75.6%
75.68%
75.77%
75.85%
75.93%
76.02%
76.1%
76.18%
76.27%
76.35%
76.43%
76.51%
76.6%
76.68%
76.76%
76.85%
76.93%
77.01%
77.1%
77.18%
77.26%
77.34%
77.43%
77.51%
77.59%
77.68%
77.76%
77.84%
77.93%
78.01%
78.09%
78.17%
78.26%
78.34%
78.42%
78.51%
78.59%
78.67%
78.76%
78.84%
78.92%
79.0%
79.09%
79.17%
79.25%
79.34%
79.42%
79.5%
79.59%
79.67%
79.75%
79.83%
79.92%
80.0%
80.08%
80.17%
80.25%
80.33%
80.41%
80.5%
80.58%
80.66%
80.75%
80.83%
80.91%
81.0%
81.08%
81.16%
81.24%
81.33%
81.41%
81.49%
81.58%
81.66%
81.74%
81.83%
81.91%
81.99%
82.07%
82.16%
82.24%
82.32%
82.41%
82.49%
82.57%
82.66%
82.74%
82.82%
82.9%
82.99%
83.07%
83.15%
83.24%
83.32%
83.4%
83.49%
83.57%
83.65%
83.73%
83.82%


...

93.57%
1.0
0.994897959184
0.992346938776
0.992346938776
0.989795918367
0.989795918367
0.989795918367
0.987244897959
0.989795918367
0.989795918367
0.987244897959
0.987244897959
0.987244897959
0.984693877551
0.984693877551
0.984693877551
0.984693877551
0.984693877551
0.984693877551
0.979591836735
0.982142857143
0.979591836735
0.974489795918
0.97193877551
0.981770833333
0.994594594595
0.969387755102
0.969387755102
0.969387755102
0.989189189189
0.966836734694
0.961734693878
0.964285714286
0.959183673469
0.964285714286
0.964285714286
0.968831168831
0.958974358974
0.959183673469
0.983651226158
0.95641025641
0.954081632653
0.954081632653
0.958441558442
0.951530612245
0.95618556701
0.951530612245
0.951406649616
0.948586118252
0.965333333333
0.965333333333
0.955613577023
0.962666666667
0.962666666667
0.962666666667
0.964864864865
0.96
0.96
0.943734015345
0.957333333333
0.957333333333
0.957333333333
0.957333333333
0.957104557641
0.954666666667
0.954666666667
0.954666666667
0.954666666667
0.95466

95.0%
1.0
0.988986784141
0.982378854626
0.982378854626
0.977973568282
0.977973568282
0.980176211454
0.977973568282
0.97577092511
0.977973568282
0.973568281938
0.971365638767
0.973568281938
0.969162995595
0.962555066079
0.995249406176
0.988290398126
0.992874109264
0.988123515439
0.981264637002
0.974358974359
0.976247030879
0.978622327791
0.976247030879
0.971496437055
0.969121140143
0.973871733967
0.972431077694
0.972431077694
0.965087281796
0.957711442786
0.991780821918
0.973333333333
0.961240310078
0.970666666667
0.972144846797
0.96
0.96
0.969359331476
0.980113636364
0.971590909091
0.965909090909
0.919799498747
0.917910447761
0.952646239554
0.972972972973
0.918158567775
0.904761904762
0.951428571429
0.949852507375
0.917112299465
0.914438502674
0.909090909091
0.862155388471
0.867924528302
0.847117794486
0.858288770053
0.813432835821
0.977064220183
0.801507537688
0.8
0.798994974874
0.80310880829
0.796915167095
0.799485861183
0.799485861183
0.799485861183
0.794344473008
0.793969849246
0.7

96.43%
1.0
0.964383561644
0.961643835616
0.982808022923
0.958448753463
0.941002949853
0.917808219178
0.86301369863
1.0
0.868493150685
0.838356164384
0.824657534247
0.861290322581
0.85993485342
0.851612903226
0.84375
0.87109375
0.892857142857
0.79781420765
0.768802228412
0.789617486339
0.760989010989
0.758241758242
0.777472527473
{'440206088': 0.7582417582417582, '1410442020': 0.9828080229226361, '1410441968': 0.84375, '922321112': 0.760989010989011, '1410441928': 0.8928571428571429, '1239394454': 0.8612903225806452, '1239394456': 0.8599348534201955, '1239394418': 0.8516129032258064, '1354635623': 0.7688022284122563, '1239394436': 0.9643835616438357, '922321224': 0.9616438356164384, '1410441766': 0.7896174863387978, '269116368': 0.8246575342465754, '1410441918': 0.87109375, '1410441750': 0.9178082191780822, '1410441970': 0.8684931506849315, '1410442058': 0.7978142076502732, '1410441854': 0.9584487534626038, '1410441936': 0.7774725274725275, '1410441858': 1.0, '12863141': 0.9410029498525

98.57%
1.0
0.9675
0.902255639098
0.899749373434
0.899749373434
0.8975
0.8975
0.897243107769
0.897243107769
0.897243107769
0.897243107769
0.897243107769
0.897243107769
0.895
0.895
0.895
0.895
0.895
0.895
0.895
0.896984924623
0.895
0.895
0.8925
0.8925
0.8925
0.8925
0.894736842105
0.8925
0.892230576441
0.887218045113
0.8925
0.89
0.89
0.89
0.8925
0.8925
0.8925
0.8925
0.8925
0.89
0.89
0.89
0.892230576441
0.892230576441
0.892230576441
0.892230576441
0.892230576441
0.892230576441
0.892230576441
0.89
0.89
0.89
0.89
0.89
0.89
0.89
0.89
0.89
0.89
0.89
0.8875
0.891959798995
0.891959798995
0.89
0.89
0.89
0.885
0.89
0.89
0.89
0.89
0.8875
0.8875
0.885
0.889447236181
0.889724310777
0.889724310777
0.8875
0.8875
0.8875
0.8825
0.8875
0.8875
0.894871794872
0.894871794872
0.894871794872
0.894871794872
0.894871794872
0.894871794872
0.8875
0.8875
0.8875
0.8875
0.886934673367
0.886934673367
0.885
0.885
0.8825
0.88
{'284521210': 0.8972431077694235, '284521214': 0.8869346733668342, '284521216': 0.8922305764411

0.949109414758
0.949109414758
0.949109414758
0.946700507614
0.946700507614
0.946700507614
0.946700507614
0.946700507614
0.944162436548
0.946564885496
0.946564885496
0.946564885496
0.944162436548
0.944020356234
0.944020356234
0.941475826972
0.948320413437
0.941624365482
0.941624365482
0.939086294416
0.941624365482
0.941475826972
0.941475826972
0.941326530612
0.939086294416
0.93893129771
0.93893129771
0.93893129771
0.93893129771
0.936386768448
0.933842239186
0.936386768448
0.936386768448
0.936386768448
0.93654822335
0.93654822335
0.936386768448
0.936386768448
0.933842239186
0.942708333333
0.942708333333
0.933842239186
0.933842239186
0.933842239186
0.931297709924
0.935400516796
0.928753180662
0.931297709924
0.928934010152
0.9375
0.934895833333
0.926208651399
0.926208651399
0.926208651399
0.926208651399
0.926208651399
0.97084548105
0.996875
0.923664122137
0.923664122137
0.923664122137
0.923664122137
0.923664122137
0.923664122137
0.945652173913
0.921119592875
0.921119592875
0.921119592875
0

Sweet, now I'm going to move all the important outputs into a folder just to keep things clean

In [8]:
mkdir GI_outputs
mv choose_mult.txt GI_outputs
mv final_GIs.txt GI_outputs
mv these_seqs_overlap.txt GI_outputs
mv these_seqs_overlap_cluster.txt GI_outputs
mv mito_COI.fa GI_outputs

In [9]:
mkdir other_outputs
mv *_resolved* other_outputs
mv seqs_to_be_blasted.txt other_outputs
mv multiple_gene_choices.txt other_outputs

In [10]:
cd GI_outputs

Since most of our ouputs are in GI format, we need to pull down the sequences themselves using a script I have.

Here's what needs to be done for each output file:

1. final_GI.txt - split by Genes and pull down sequences
2. choose_mult - these are the ones that we couldn't choose between for cluster analysis (ie they are only a few nts different and same length)
3. these_seqs_overlap.txt - these sequences don't align togther but align to the same part of the sequence, so something is wierd. 
4. these_seqs_overlap_cluster.txt - same as 3 but from cluster analysis
5. mito_COI.fa - nothing, these are the sequences that got pulled from areas of the mitochondria

In [12]:
mv ../blast_results.db .
./splitGIs.py final_GIs.txt blast_results.db

In [13]:
for i in *_GIs.txt; do ./getseqsfromGenbank.py $i; done

100.0%
70.67%
100.0%
100.0%
100.0%
20.12%
40.24%
60.36%
80.48%
100.0%
100.0%
38.99%
77.97%
100.0%
5.39%
10.78%
16.17%
21.56%
26.95%
32.34%
37.73%
43.12%
48.5%
53.89%
59.28%
64.67%
70.06%
75.45%
80.84%
86.23%
91.62%
97.01%
100.0%
67.34%
100.0%
100.0%
98.52%
100.0%
87.34%
100.0%
100.0%
61.16%
100.0%
44.35%
88.69%
100.0%


In [14]:
mkdir final_seqs_fasta
mv *.fa final_seqs_fasta

(removed the _GI.txt)
Now I'm going to do the same thing with the choose_mult - so I can keep the cleaned and multiple sequences separate for now

In [15]:
./splitGIs.py choose_mult.txt blast_results.db

In [16]:
for i in *_GIs.txt; do ./getseqsfromGenbank.py $i; done

100.0%
100.0%
100.0%
100.0%
34.25%
68.49%
100.0%
100.0%
100.0%
5.39%
10.78%
16.17%
21.56%
26.95%
32.34%
37.73%
43.12%
48.5%
53.89%
59.28%
64.67%
70.06%
75.45%
80.84%
86.23%
91.62%
97.01%
100.0%
100.0%
100.0%
100.0%
100.0%
100.0%
100.0%
100.0%


In [17]:
mkdir choose_mult_fasta
mv *_GIs.fa choose_mult_fasta

For the overlapping sequences, I looked at each group to determine if I could choose which sequence to choose. For those that were too similar to choose, I added them to the choose_multiple group. For those that were just too messy to choose, I ignored them.

First I'm going to try and figure out how to choose between those with multiple choices - I'm not sure how to go about this.

In [4]:
cd choose_mult_fasta
for i in *.fa; do ./rename_seqs.py $i ../blast_results.db ../../NAButterflies_wsyns_2.db; done

0.111111111111
0.222222222222
0.333333333333
0.444444444444
0.555555555556
0.666666666667
0.777777777778
0.888888888889
1.0
0.111111111111
Traceback (most recent call last):
  File "./rename_seqs.py", line 27, in <module>
    sp_tc_id = tc_id
NameError: name 'tc_id' is not defined
0.0555555555556
0.111111111111
0.166666666667
0.222222222222
0.277777777778
0.333333333333
0.388888888889
0.444444444444
0.5
0.555555555556
0.611111111111
0.666666666667
0.722222222222
0.777777777778
0.833333333333
0.888888888889
0.944444444444
1.0
0.25
0.5
0.75
1.0
0.5
1.0
0.00171232876712
0.00342465753425
0.00513698630137
0.00684931506849
0.00856164383562
0.0102739726027
0.0119863013699
0.013698630137
0.0154109589041
0.0171232876712
0.0188356164384
0.0205479452055
0.0222602739726
0.0239726027397
0.0256849315068
0.027397260274
0.0291095890411
0.0308219178082
0.0325342465753
0.0342465753425
0.0359589041096
0.0376712328767
0.0393835616438
0.041095890411
0.0428082191781
0.0445205479452
0.0462328767123
0.0479452

0.880136986301
0.881849315068
0.883561643836
0.885273972603
0.88698630137
0.888698630137
0.890410958904
0.892123287671
0.893835616438
0.895547945205
0.897260273973
0.89897260274
0.900684931507
0.902397260274
0.904109589041
0.905821917808
0.907534246575
0.909246575342
0.91095890411
0.912671232877
0.914383561644
0.916095890411
0.917808219178
0.919520547945
0.921232876712
0.922945205479
0.924657534247
0.926369863014
0.928082191781
0.929794520548
0.931506849315
0.933219178082
0.934931506849
0.936643835616
0.938356164384
0.940068493151
0.941780821918
0.943493150685
0.945205479452
0.946917808219
0.948630136986
0.950342465753
0.952054794521
0.953767123288
0.955479452055
0.957191780822
0.958904109589
0.960616438356
0.962328767123
0.96404109589
0.965753424658
0.967465753425
0.969178082192
0.970890410959
0.972602739726
0.974315068493
0.97602739726
0.977739726027
0.979452054795
0.981164383562
0.982876712329
0.984589041096
0.986301369863
0.98801369863
0.989726027397
0.991438356164
0.993150684932
0

I'm going to use a script to create a consensus sequence of those that have 95% or more identity. Any place with mismatching sequences will have an N instead. 

In [11]:
for i in *_renamed.fa; do ./overlap_dups.py $i; done

Now I'm going to look at the ones that didn't get cleaned up - if they are just so different and one of them is not clearly wrong when blasting, I'm deleting them. Some are considered duplicates because they align to different parts of the locus. I did this by hand since there weren't that many.

Next I'm looking at the sequences that are not duplicates. First we must concantenate the mito file and the regular file, then rename the sequences, then merge the ones that have overlapping parts. I lowered the identity threshold to 80% because of the shorter length of overlap that COI tends to have. The ones that don't meet this threshold I'm keeping because they align to different parts of the locus. 

In [12]:
cd ../final_seqs_fasta

In [16]:
cat mito_COI.fa COI_trnL_COII_GIs.fa > COI_trnL_COII_mito_GIs.fa

In [17]:
for i in *.fa; do ./rename_seqs.py $i ../blast_results.db ../../NAButterflies_wsyns_2.db; done

0.00613496932515
0.0122699386503
0.0184049079755
0.0245398773006
0.0306748466258
0.0368098159509
0.0429447852761
0.0490797546012
0.0552147239264
0.0613496932515
0.0674846625767
0.0736196319018
0.079754601227
0.0858895705521
0.0920245398773
0.0981595092025
0.104294478528
0.110429447853
0.116564417178
0.122699386503
0.128834355828
0.134969325153
0.141104294479
0.147239263804
0.153374233129
0.159509202454
0.165644171779
0.171779141104
0.177914110429
0.184049079755
0.19018404908
0.196319018405
0.20245398773
0.208588957055
0.21472392638
0.220858895706
0.226993865031
0.233128834356
0.239263803681
0.245398773006
0.251533742331
0.257668711656
0.263803680982
0.269938650307
0.276073619632
0.282208588957
0.288343558282
0.294478527607
0.300613496933
0.306748466258
0.312883435583
0.319018404908
0.325153374233
0.331288343558
0.337423312883
0.343558282209
0.349693251534
0.355828220859
0.361963190184
0.368098159509
0.374233128834
0.38036809816
0.386503067485
0.39263803681
0.398773006135
0.40490797546


0.0749027237354
0.0758754863813
0.0768482490272
0.0778210116732
0.0787937743191
0.079766536965
0.0807392996109
0.0817120622568
0.0826848249027
0.0836575875486
0.0846303501946
0.0856031128405
0.0865758754864
0.0875486381323
0.0885214007782
0.0894941634241
0.09046692607
0.091439688716
0.0924124513619
0.0933852140078
0.0943579766537
0.0953307392996
0.0963035019455
0.0972762645914
0.0982490272374
0.0992217898833
0.100194552529
0.101167315175
0.102140077821
0.103112840467
0.104085603113
0.105058365759
0.106031128405
0.107003891051
0.107976653696
0.108949416342
0.109922178988
0.110894941634
0.11186770428
0.112840466926
0.113813229572
0.114785992218
0.115758754864
0.11673151751
0.117704280156
0.118677042802
0.119649805447
0.120622568093
0.121595330739
0.122568093385
0.123540856031
0.124513618677
0.125486381323
0.126459143969
0.127431906615
0.128404669261
0.129377431907
0.130350194553
0.131322957198
0.132295719844
0.13326848249
0.134241245136
0.135214007782
0.136186770428
0.137159533074
0.1381

0.609922178988
0.610894941634
0.61186770428
0.612840466926
0.613813229572
0.614785992218
0.615758754864
0.61673151751
0.617704280156
0.618677042802
0.619649805447
0.620622568093
0.621595330739
0.622568093385
0.623540856031
0.624513618677
0.625486381323
0.626459143969
0.627431906615
0.628404669261
0.629377431907
0.630350194553
0.631322957198
0.632295719844
0.63326848249
0.634241245136
0.635214007782
0.636186770428
0.637159533074
0.63813229572
0.639105058366
0.640077821012
0.641050583658
0.642023346304
0.642996108949
0.643968871595
0.644941634241
0.645914396887
0.646887159533
0.647859922179
0.648832684825
0.649805447471
0.650778210117
0.651750972763
0.652723735409
0.653696498054
0.6546692607
0.655642023346
0.656614785992
0.657587548638
0.658560311284
0.65953307393
0.660505836576
0.661478599222
0.662451361868
0.663424124514
0.66439688716
0.665369649805
0.666342412451
0.667315175097
0.668287937743
0.669260700389
0.670233463035
0.671206225681
0.672178988327
0.673151750973
0.674124513619
0.6

0.189083820663
0.191033138402
0.19298245614
0.194931773879
0.196881091618
0.198830409357
0.200779727096
0.202729044834
0.204678362573
0.206627680312
0.208576998051
0.210526315789
0.212475633528
0.214424951267
0.216374269006
0.218323586745
0.220272904483
0.222222222222
0.224171539961
0.2261208577
0.228070175439
0.230019493177
0.231968810916
0.233918128655
0.235867446394
0.237816764133
0.239766081871
0.24171539961
0.243664717349
0.245614035088
0.247563352827
0.249512670565
0.251461988304
0.253411306043
0.255360623782
0.25730994152
0.259259259259
0.261208576998
0.263157894737
0.265107212476
0.267056530214
0.269005847953
0.270955165692
0.272904483431
0.27485380117
0.276803118908
0.278752436647
0.280701754386
0.282651072125
0.284600389864
0.286549707602
0.288499025341
0.29044834308
0.292397660819
0.294346978558
0.296296296296
0.298245614035
0.300194931774
0.302144249513
0.304093567251
0.30604288499
0.307992202729
0.309941520468
0.311890838207
0.313840155945
0.315789473684
0.317738791423
0.3

0.451178451178
0.454545454545
0.457912457912
0.461279461279
0.464646464646
0.468013468013
0.47138047138
0.474747474747
0.478114478114
0.481481481481
0.484848484848
0.488215488215
0.491582491582
0.494949494949
0.498316498316
0.501683501684
0.505050505051
0.508417508418
0.511784511785
0.515151515152
0.518518518519
0.521885521886
0.525252525253
0.52861952862
0.531986531987
0.535353535354
0.538720538721
0.542087542088
0.545454545455
0.548821548822
0.552188552189
0.555555555556
0.558922558923
0.56228956229
0.565656565657
0.569023569024
0.572390572391
0.575757575758
0.579124579125
0.582491582492
0.585858585859
0.589225589226
0.592592592593
0.59595959596
0.599326599327
0.602693602694
0.606060606061
0.609427609428
0.612794612795
0.616161616162
0.619528619529
0.622895622896
0.626262626263
0.62962962963
0.632996632997
0.636363636364
0.639730639731
0.643097643098
0.646464646465
0.649831649832
0.653198653199
0.656565656566
0.659932659933
0.6632996633
0.666666666667
0.670033670034
0.673400673401
0.

0.786026200873
0.7903930131
0.794759825328
0.799126637555
0.803493449782
0.807860262009
0.812227074236
0.816593886463
0.82096069869
0.825327510917
0.829694323144
0.834061135371
0.838427947598
0.842794759825
0.847161572052
0.851528384279
0.855895196507
0.860262008734
0.864628820961
0.868995633188
0.873362445415
0.877729257642
0.882096069869
0.886462882096
0.890829694323
0.89519650655
0.899563318777
0.903930131004
0.908296943231
0.912663755459
0.917030567686
0.921397379913
0.92576419214
0.930131004367
0.934497816594
0.938864628821
0.943231441048
0.947598253275
0.951965065502
0.956331877729
0.960698689956
0.965065502183
0.96943231441
0.973799126638
0.978165938865
0.982532751092
0.986899563319
0.991266375546
0.995633187773
1.0
0.00613496932515
0.0122699386503
0.0184049079755
0.0245398773006
0.0306748466258
0.0368098159509
0.0429447852761
0.0490797546012
0.0552147239264
0.0613496932515
0.0674846625767
0.0736196319018
0.079754601227
0.0858895705521
0.0920245398773
0.0981595092025
0.104294478

0.019955654102
0.0221729490022
0.0243902439024
0.0266075388027
0.0288248337029
0.0310421286031
0.0332594235033
0.0354767184035
0.0376940133038
0.039911308204
0.0421286031042
0.0443458980044
0.0465631929047
0.0487804878049
0.0509977827051
0.0532150776053
0.0554323725055
0.0576496674058
0.059866962306
0.0620842572062
0.0643015521064
0.0665188470067
0.0687361419069
0.0709534368071
0.0731707317073
0.0753880266075
0.0776053215078
0.079822616408
0.0820399113082
0.0842572062084
0.0864745011086
0.0886917960089
0.0909090909091
0.0931263858093
0.0953436807095
0.0975609756098
0.09977827051
0.10199556541
0.10421286031
0.106430155211
0.108647450111
0.110864745011
0.113082039911
0.115299334812
0.117516629712
0.119733924612
0.121951219512
0.124168514412
0.126385809313
0.128603104213
0.130820399113
0.133037694013
0.135254988914
0.137472283814
0.139689578714
0.141906873614
0.144124168514
0.146341463415
0.148558758315
0.150776053215
0.152993348115
0.155210643016
0.157427937916
0.159645232816
0.161862527

In [24]:
for i in *_renamed.fa; do ./overlap_dups.py $i; done

In [26]:
cd ../
mkdir align_seqs

Now to add the sequences from the choose_mult and start aligning!

In [29]:
cat ./final_seqs_fasta/ArgKin_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/ArgKin_GIs_renamed_dupsmerge.fas > align_seqs/ArgKin_all.fa
cat ./final_seqs_fasta/CAD_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/CAD_GIs_renamed_dupsmerge.fas > align_seqs/CAD_all.fa
cat ./final_seqs_fasta/CAT_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/CAT_GIs_renamed_dupsmerge.fas > align_seqs/CAT_all.fa
cat ./final_seqs_fasta/CoA_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/CoA_GIs_renamed_dupsmerge.fas > align_seqs/CoA_all.fa
cat ./final_seqs_fasta/COI_trnL_COII_mito_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/COI_trnL_COII_GIs_renamed_dupsmerge.fas > align_seqs/COI_trnL_COII_all.fa
cat ./final_seqs_fasta/DDC_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/DDC_GIs_renamed_dupsmerge.fas > align_seqs/DDC_all.fa
cat ./final_seqs_fasta/EF-1a_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/EF-1a_GIs_renamed_dupsmerge.fas > align_seqs/EF-1a_all.fa
cat ./final_seqs_fasta/GAPDH_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/GAPDH_GIs_renamed_dupsmerge.fas > align_seqs/GAPDH_all.fa
cat ./final_seqs_fasta/HCL_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/HCL_GIs_renamed_dupsmerge.fas > align_seqs/HCL_all.fa
cat ./final_seqs_fasta/IDH_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/IDH_GIs_renamed_dupsmerge.fas > align_seqs/IDH_all.fa
cat ./final_seqs_fasta/MDH_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/MDH_GIs_renamed_dupsmerge.fas > align_seqs/MDH_all.fa
cat ./final_seqs_fasta/RpS2_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/RpS2_GIs_renamed_dupsmerge.fas > align_seqs/RpS2_all.fa
cat ./final_seqs_fasta/RpS5_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/RpS5_GIs_renamed_dupsmerge.fas > align_seqs/RpS5_all.fa
cat ./final_seqs_fasta/Wgl_GIs_renamed_dupsmerge.fas ./choose_mult_fasta/Wgl_GIs_renamed_dupsmerge.fas > align_seqs/Wgl_all.fa


In [32]:
cd align_seqs
for i in *.fa; do python LONGREF_ALIGN.py $i 2> /dev/null; done

ArgKin_all_FULL.fas:1 seqs
ArgKin_all_FRAGS.fas: 166 seqs

RUNNING 1 FULL, LOTS FRAGS
CAD_all_FULL.fas:25 seqs
CAD_all_FRAGS.fas: 265 seqs

RUNNING FEW FULL, LOTS FRAGS
CAT_all_FULL.fas:5 seqs
CAT_all_FRAGS.fas: 12 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT.....
RUNNING MAFFT ADDING FRAGS.....
CoA_all_FULL.fas:1 seqs
CoA_all_FRAGS.fas: 12 seqs

RUNNING 1 FULL, FEW FRAGS
COI_trnL_COII_all_FULL.fas:619 seqs
COI_trnL_COII_all_FRAGS.fas: 447 seqs

RUNNING LOTS FULL, LOTS FRAGS
0
1
2
3
4
5
6
7
8
9
10
11
12
0
1
2
3
4
5
6
7
8
DDC_all_FULL.fas:12 seqs
DDC_all_FRAGS.fas: 41 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT.....
RUNNING MAFFT ADDING FRAGS.....
EF-1a_all_FULL.fas:124 seqs
EF-1a_all_FRAGS.fas: 418 seqs

RUNNING LOTS FULL, LOTS FRAGS
0
1
2
0
1
2
3
4
5
6
7
8
GAPDH_all_FULL.fas:1 seqs
GAPDH_all_FRAGS.fas: 314 seqs

RUNNING 1 FULL, LOTS FRAGS
HCL_all_FULL.fas:10 seqs
HCL_all_FRAGS.fas: 1 seqs

RUNNING FEW FULL, FEW FRAGS
RUNNING MAFFT FULL ALIGNMENT...

Now we're going to move each final alignment to a new file and check the alignments

In [33]:
mkdir aligned_files
mv Wgl_all_aligned10.fas aligned_files
mv RpS5_all_aligned7.fas aligned_files
mv RpS2_all_aligned4.fas aligned_files
mv MDH_all_aligned5.fas aligned_files
mv IDH_all_aligned5.fas aligned_files
mv HCL_all_FINALALIGN.fa aligned_files
mv GAPDH_all_aligned7.fas aligned_files
mv EF-1a_all_FINALALIGN.fa aligned_files
mv DDC_all_FINALALIGN.fa aligned_files
mv COI_trnL_COII_all_FINALALIGN.fa aligned_files
mv CoA_all_FINALALIGN.fa aligned_files
mv CAT_all_FINALALIGN.fa aligned_files
mv CAD_all_aligned6.fas aligned_files
mv ArgKin_all_aligned4.fas aligned_files

Now since there are inserts, I'm going to use a threshold to remove sites that are 95% or more gaps and then realign using mafft.

In [34]:
cd aligned_files

In [36]:
for i in *.fa*; do python remove_sites_threshold.py -i $i -t .95; done

In [37]:
for i in *95.fas; do mafft $i > ${i/%.fas/_aligned.fas} 2> /dev/null; done

Next I'm going to use Aliview to translate to amino acids, find the reading frame, and chop off the extra flanks where the 'messiness' starts. I'm being conservative about chopping, since when these get aligned to longer sequences, we can clean these up more.

The last step is to rename them again since mafft adds prefixes to sequences that have been reversed and then merge the sequences that are fragments.

I need to redo IDH - had to remove Papilio Xuthus.

In [39]:
mkdir final_align
for i in *_edit.fas; do mv $i final_align; done

mkdir: cannot create directory ‘final_align’: File exists


In [40]:
cd final_align
for i in *.fas; do sed -i 's/_R_//g' $i; done

In [43]:
for i in *.fa*; do ./overlap_dups.py $i; done

In [44]:
mkdir final_align
mv *_dupsmerge.fas final_align

Some of the COI sequences weren't aligning correctly, so I blasted them and figured out they were fish. So I pulled down the longest sequences from each species and then realigned.

In [45]:
cd final_align

In [46]:
mv ArgKin_all_aligned4_threshold95_aligned_edit_dupsmerge.fas ArgKin_GB.fa
mv CAD_all_aligned6_threshold95_edit_dupsmerge.fas CAD_GB.fa
mv CAT_all_FINALALIGN_threshold95_aligned_edit_dupsmerge.fas CAT_GB.fa
mv CoA_all_FINALALIGN_threshold95_aligned_edit_dupsmerge.fas CoA_GB.fa
mv COI_trnL_COII_all2.fas COI_trnL_COII_GB.fa
mv DDC_all_FINALALIGN_edit_dupsmerge.fas DDC_GB.fa
mv EF-1a_all_FINALALIGN_threshold95_aligned_edit_dupsmerge.fas EF-1a_GB.fa
mv GAPDH_all_aligned7_threshold95_aligned_edit_dupsmerge.fas GAPDH_GB.fa
mv HCL_all_FINALALIGN_threshold95_aligned_edit_dupsmerge.fas HCL_GB.fa
mv IDH_all_aligned5_edit_dupsmerge.fas IDH_GB.fa
mv MDH_all_aligned5_threshold95_aligned_edit_dupsmerge.fas MDH_GB.fa
mv RpS2_all_aligned4_threshold95_aligned_edit_dupsmerge.fas RpS2_GB.fa
mv RpS5_all_aligned7_threshold95_aligned_edit_dupsmerge.fas RpS5_GB.fa
mv Wgl_all_aligned10_threshold95_aligned_edit_dupsmerge.fas Wgl_GB.fa

I realized that the names of the sequences are the synonym names. So I'm going to rename and I might have to merge some again.

In [60]:
for i in *.fa; do ./rename_seqs.py $i NAButterflies_wsyns_2.db; done

0.0059880239521
0.0119760479042
0.0179640718563
0.0239520958084
0.0299401197605
0.0359281437126
0.0419161676647
0.0479041916168
0.0538922155689
0.059880239521
0.0658682634731
0.0718562874251
0.0778443113772
0.0838323353293
0.0898203592814
0.0958083832335
0.101796407186
0.107784431138
0.11377245509
0.119760479042
0.125748502994
0.131736526946
0.137724550898
0.14371257485
0.149700598802
0.155688622754
0.161676646707
0.167664670659
0.173652694611
0.179640718563
0.185628742515
0.191616766467
0.197604790419
0.203592814371
0.209580838323
0.215568862275
0.221556886228
0.22754491018
0.233532934132
0.239520958084
0.245508982036
0.251497005988
0.25748502994
0.263473053892
0.269461077844
0.275449101796
0.281437125749
0.287425149701
0.293413173653
0.299401197605
0.305389221557
0.311377245509
0.317365269461
0.323353293413
0.329341317365
0.335329341317
0.341317365269
0.347305389222
0.353293413174
0.359281437126
0.365269461078
0.37125748503
0.377245508982
0.383233532934
0.389221556886
0.395209580838


0.0814332247557
0.0825190010858
0.0836047774159
0.0846905537459
0.085776330076
0.0868621064061
0.0879478827362
0.0890336590662
0.0901194353963
0.0912052117264
0.0922909880565
0.0933767643865
0.0944625407166
0.0955483170467
0.0966340933768
0.0977198697068
0.0988056460369
0.099891422367
0.100977198697
0.102062975027
0.103148751357
0.104234527687
0.105320304017
0.106406080347
0.107491856678
0.108577633008
0.109663409338
0.110749185668
0.111834961998
0.112920738328
0.114006514658
0.115092290988
0.116178067318
0.117263843648
0.118349619978
0.119435396308
0.120521172638
0.121606948969
0.122692725299
0.123778501629
0.124864277959
0.125950054289
0.127035830619
0.128121606949
0.129207383279
0.130293159609
0.131378935939
0.132464712269
0.133550488599
0.134636264929
0.13572204126
0.13680781759
0.13789359392
0.13897937025
0.14006514658
0.14115092291
0.14223669924
0.14332247557
0.1444082519
0.14549402823
0.14657980456
0.14766558089
0.14875135722
0.14983713355
0.150922909881
0.152008686211
0.1530944

0.678610206298
0.679695982628
0.680781758958
0.681867535288
0.682953311618
0.684039087948
0.685124864278
0.686210640608
0.687296416938
0.688382193268
0.689467969598
0.690553745928
0.691639522258
0.692725298588
0.693811074919
0.694896851249
0.695982627579
0.697068403909
0.698154180239
0.699239956569
0.700325732899
0.701411509229
0.702497285559
0.703583061889
0.704668838219
0.705754614549
0.706840390879
0.70792616721
0.70901194354
0.71009771987
0.7111834962
0.71226927253
0.71335504886
0.71444082519
0.71552660152
0.71661237785
0.71769815418
0.71878393051
0.71986970684
0.72095548317
0.722041259501
0.723127035831
0.724212812161
0.725298588491
0.726384364821
0.727470141151
0.728555917481
0.729641693811
0.730727470141
0.731813246471
0.732899022801
0.733984799131
0.735070575461
0.736156351792
0.737242128122
0.738327904452
0.739413680782
0.740499457112
0.741585233442
0.742671009772
0.743756786102
0.744842562432
0.745928338762
0.747014115092
0.748099891422
0.749185667752
0.750271444083
0.7513572

0.369202226345
0.371057513915
0.372912801484
0.374768089054
0.376623376623
0.378478664193
0.380333951763
0.382189239332
0.384044526902
0.385899814471
0.387755102041
0.38961038961
0.39146567718
0.39332096475
0.395176252319
0.397031539889
0.398886827458
0.400742115028
0.402597402597
0.404452690167
0.406307977737
0.408163265306
0.410018552876
0.411873840445
0.413729128015
0.415584415584
0.417439703154
0.419294990724
0.421150278293
0.423005565863
0.424860853432
0.426716141002
0.428571428571
0.430426716141
0.432282003711
0.43413729128
0.43599257885
0.437847866419
0.439703153989
0.441558441558
0.443413729128
0.445269016698
0.447124304267
0.448979591837
0.450834879406
0.452690166976
0.454545454545
0.456400742115
0.458256029685
0.460111317254
0.461966604824
0.463821892393
0.465677179963
0.467532467532
0.469387755102
0.471243042672
0.473098330241
0.474953617811
0.47680890538
0.47866419295
0.480519480519
0.482374768089
0.484230055659
0.486085343228
0.487940630798
0.489795918367
0.491651205937
0.

0.669841269841
0.673015873016
0.67619047619
0.679365079365
0.68253968254
0.685714285714
0.688888888889
0.692063492063
0.695238095238
0.698412698413
0.701587301587
0.704761904762
0.707936507937
0.711111111111
0.714285714286
0.71746031746
0.720634920635
0.72380952381
0.726984126984
0.730158730159
0.733333333333
0.736507936508
0.739682539683
0.742857142857
0.746031746032
0.749206349206
0.752380952381
0.755555555556
0.75873015873
0.761904761905
0.765079365079
0.768253968254
0.771428571429
0.774603174603
0.777777777778
0.780952380952
0.784126984127
0.787301587302
0.790476190476
0.793650793651
0.796825396825
0.8
0.803174603175
0.806349206349
0.809523809524
0.812698412698
0.815873015873
0.819047619048
0.822222222222
0.825396825397
0.828571428571
0.831746031746
0.834920634921
0.838095238095
0.84126984127
0.844444444444
0.847619047619
0.850793650794
0.853968253968
0.857142857143
0.860317460317
0.863492063492
0.866666666667
0.869841269841
0.873015873016
0.87619047619
0.879365079365
0.88253968254

0.0295857988166
0.0355029585799
0.0414201183432
0.0473372781065
0.0532544378698
0.0591715976331
0.0650887573964
0.0710059171598
0.0769230769231
0.0828402366864
0.0887573964497
0.094674556213
0.100591715976
0.10650887574
0.112426035503
0.118343195266
0.12426035503
0.130177514793
0.136094674556
0.14201183432
0.147928994083
0.153846153846
0.159763313609
0.165680473373
0.171597633136
0.177514792899
0.183431952663
0.189349112426
0.195266272189
0.201183431953
0.207100591716
0.213017751479
0.218934911243
0.224852071006
0.230769230769
0.236686390533
0.242603550296
0.248520710059
0.254437869822
0.260355029586
0.266272189349
0.272189349112
0.278106508876
0.284023668639
0.289940828402
0.295857988166
0.301775147929
0.307692307692
0.313609467456
0.319526627219
0.325443786982
0.331360946746
0.337278106509
0.343195266272
0.349112426036
0.355029585799
0.360946745562
0.366863905325
0.372781065089
0.378698224852
0.384615384615
0.390532544379
0.396449704142
0.402366863905
0.408284023669
0.414201183432
0.

0.0739219712526
0.0759753593429
0.0780287474333
0.0800821355236
0.082135523614
0.0841889117043
0.0862422997947
0.088295687885
0.0903490759754
0.0924024640657
0.0944558521561
0.0965092402464
0.0985626283368
0.100616016427
0.102669404517
0.104722792608
0.106776180698
0.108829568789
0.110882956879
0.112936344969
0.11498973306
0.11704312115
0.11909650924
0.121149897331
0.123203285421
0.125256673511
0.127310061602
0.129363449692
0.131416837782
0.133470225873
0.135523613963
0.137577002053
0.139630390144
0.141683778234
0.143737166324
0.145790554415
0.147843942505
0.149897330595
0.151950718686
0.154004106776
0.156057494867
0.158110882957
0.160164271047
0.162217659138
0.164271047228
0.166324435318
0.168377823409
0.170431211499
0.172484599589
0.17453798768
0.17659137577
0.17864476386
0.180698151951
0.182751540041
0.184804928131
0.186858316222
0.188911704312
0.190965092402
0.193018480493
0.195071868583
0.197125256674
0.199178644764
0.201232032854
0.203285420945
0.205338809035
0.207392197125
0.209

In [62]:
./overlap_dups.py COI_trnL_COII_GB_renamed.fa

Now to go through and look at them individually and make sure it still looks okay

In [63]:
mkdir final_valid
cp ArgKin_GB_renamed.fa final_valid/ArgKin_GB.fas
cp CAD_GB_renamed.fa final_valid/CAD_GB.fas
cp CAT_GB_renamed.fa final_valid/CAT_GB.fas
cp CoA_GB_renamed.fa final_valid/CoA_GB.fas
cp COI_trnL_COII_GB_renamed_dupsmerge.fas final_valid/COI_trnL_COII_GB.fas
cp DDC_GB_renamed.fa final_valid/DDC_GB.fas
cp EF-1a_GB_renamed.fa final_valid/EF-1a_GB.fas
cp GAPDH_GB_renamed.fa final_valid/GAPDH_GB.fas
cp HCL_GB_renamed.fa final_valid/HCL_GB.fas
cp IDH_GB_renamed.fa final_valid/IDH_GB.fas
cp MDH_GB_renamed.fa final_valid/MDH_GB.fas
cp RpS2_GB_renamed.fa final_valid/RpS2_GB.fas
cp RpS5_GB_renamed.fa final_valid/RpS5_GB.fas
cp Wgl_GB_renamed.fa final_valid/Wgl_GB.fas

And now we're done..